This exercise will require you to pull some data from https://data.nasdaq.com/ (formerly Quandl API).

As a first step, you will need to register a free account on the https://data.nasdaq.com/ website.

After you register, you will be provided with a unique API key, that you should store:

*Note*: Use a `.env` file and put your key in there and `python-dotenv` to access it in this notebook. 

The code below uses a key that was used when generating this project but has since been deleted. Never submit your keys to source control. There is a `.env-example` file in this repository to illusrtate what you need. Copy that to a file called `.env` and use your own api key in that `.env` file. Make sure you also have a `.gitignore` file with a line for `.env` added to it. 

The standard Python gitignore is [here](https://github.com/github/gitignore/blob/master/Python.gitignore) you can just copy that. 

In [1]:
# get api key from your .env file
import os
from dotenv import load_dotenv

load_dotenv()
API_KEY = os.getenv('NASDAQ_API_KEY')

Nasdaq Data has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Nasdaq Data API instructions here: https://docs.data.nasdaq.com/docs/in-depth-usage

While there is a dedicated Python package for connecting to the Nasdaq API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests
import collections
import json


Note: API's can change a bit with each version, for this exercise it is reccomended to use the nasdaq api at `https://data.nasdaq.com/api/v3/`. This is the same api as what used to be quandl so `https://www.quandl.com/api/v3/` should work too.

Hint: We are looking for the `AFX_X` data on the `datasets/FSE/` dataset.

In [3]:
# Now, call the Nasdaq API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned

r = requests.get('https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X?start_date=2012-09-20&end_date=2012-09-21&api_key=' + API_KEY)
r.json();

In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
r = requests.get('https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X?start_date=2017-01-01&end_date=2017-12-31&api_key=' + API_KEY)
r_json = r.json();

In [5]:
r_dataset = dict(r.json())

r_keys = r_dataset['dataset']['column_names']

r_data=r_dataset['dataset']['data']



In [6]:

afx_ld = [ dict (zip(r_keys, i)) for i in r_data ]
type(afx_ld[0]['Open'])


float

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [7]:
opens = []
for i in range(len(afx_ld)):
    if afx_ld[i]['Open'] != None:
        tup = (afx_ld[i]['Open'], afx_ld[i]['Date'])
        opens.append(tup)


In [8]:
hi = max(opens)
lo = min(opens)
print(hi, lo)

(53.11, '2017-12-14') (34.0, '2017-01-24')


In [9]:
change = []
for i in range(len(afx_ld)):
    if afx_ld[i]['High'] and afx_ld[i]['Low'] != None:
        day_change = afx_ld[i]['High'] - afx_ld[i]['Low']
        tup = (abs(day_change), afx_ld[i]['Date'], day_change)
        change.append(tup)

In [10]:
max_change = max(change)
print(max_change)

(2.8100000000000023, '2017-05-11', 2.8100000000000023)


In [11]:
change_close = []
for i in range(len(afx_ld)):
    if afx_ld[i]['Close'] and afx_ld[i-1]['Close'] != None:
        days_change = afx_ld[i]['Close'] - afx_ld[i-1]['Close']
        tup = (abs(days_change), afx_ld[i]['Date'], afx_ld[i-1]['Date'], days_change)
        change_close.append(tup)


In [12]:
max_change_close = (max(change_close[1:]))
print(max_change_close)

(2.559999999999995, '2017-08-08', '2017-08-09', 2.559999999999995)


In [13]:
volume = []
days=0
for i in range(len(afx_ld)):
    if afx_ld[i]['Traded Volume'] != None:
        volume.append(afx_ld[i]['Traded Volume'])
        days += 1
average = sum(volume)/days

In [14]:
average

89124.33725490196

In [15]:
median_vol = []
for i in range(len(afx_ld)):
    if afx_ld[i]['Traded Volume'] != None:
        tup = (afx_ld[i]['Traded Volume'], afx_ld[i]['Date'])
        median_vol.append(tup)

In [16]:
median_vol_srt = sorted(median_vol)
median = median_vol_srt[128]
print(median)

(76600.0, '2017-09-27')
